문장요약

In [1]:
import requests
from bs4 import BeautifulSoup

from konlpy.tag import Mecab
mecab = Mecab()

def get_news(url):
    headers = {'user-agent':'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    
    article=html.select_one("#articleBodyContents").text.strip()
    
    return article

url='https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'

news_article = get_news(url)

In [ ]:
from nltk.tokenize import sent_tokenize #이걸로 문장 분리

Luhn Summarize

In [2]:
#kss 활용
from kss import split_sentences

def get_sentences(text):
    #return sent_tokenize(text)
    return split_sentences(text) #text에 있는 문장 토큰화

def get_words(text,isNoun=False):
    if isNoun: #명사만 가져오기
        return [token[0] for token in mecab.pos(text) if token[1][0]=='N' and len(token[0]) > 0]
    else: #아니면 형태소로 나누기만 하기
        return [token[0] for token in mecab.pos(text)]

In [3]:
#함수로 분리
sentence_list = get_sentences(news_article)
print(len(sentence_list))
sentence_list[:3]

[Korean Sentence Splitter]: Initializing Pynori...


23


['과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다.',
 '새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.',
 '22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다.']

In [4]:
#kss로 문장 분리하기; kss는 한국어 문장 분리기 중 성능 좋지만 시간 오래 걸림
from kss import split_sentences
sentence_list = split_sentences(news_article)
print(len(sentence_list))
print(sentence_list)

23
['과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다.', '새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.', '22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다.', '유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.', '◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다.', '문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.', '이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다.', '과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.', '대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다.', '이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는

In [5]:
#중요단어 계산: 중복허용 안되므로 set ==>빈도수 계산
def get_keywords(word_list,min_ratio=0.001,max_ratio=0.5):
    keywords = set() #중요단어를 set에 넣기
    
    count_dict={}
    
    for word in word_list:
        if word in count_dict.keys():
            count_dict[word] = count_dict[word]+1 #이미 단어 있으면 빈도수 +1
        else:
            count_dict[word] = 1 #아직 단어 없으면 빈도 1로 변경

    for word,cnt in count_dict.items():
        #모든 word count에 있는 애들 다 돌기 위함. count_dict는 딕셔너리이므로 key와 value 반환
        word_percentage = cnt / len(word_list)
        #전체단어 중 해당 단어가 몇번 등장하는지(0.001부터 0.5까지)
        #cnt = count_dict[word] ==>해당 단어의 등장빈도 수
        
        #min-max 사이에 있으면 중요 키워드로 하겠다
        if word_percentage >= min_ratio and word_percentage <= max_ratio:
            keywords.add(word) #set이니까 add
    return keywords

In [6]:
get_keywords(['바나나','사과','바나나','바나나','포도'])
#바나나는 0.5이상으로 너무 자주 등장해서 키워드에 포함x

{'사과', '포도'}

In [7]:
#문장 중요도 계산; sentence 내에서 키워드 있는지 검사 ex.4^2/7 검사하기
def get_sentence_weight(token_list, keywords):
    window_start=0; window_end =-1
    
    for i in range(len(token_list)): #token 개수만큼 돌기
        if token_list[i] in keywords:
            window_start=i #중요단어 등장하는 시작점 찾기
            break
            
    for i in range(len(token_list)-1,-1,-1): #거꾸로 range,0도 포함하기 위해 두번째에 -1
        if token_list[i] in keywords:
            window_end=i #중요단어 끝지점 찾기
            break
            
    if window_start > window_end:
        return 0 #해당 문장에 대한 weight를 0으로 줌
    
    window_size = window_end - window_start + 1
    
    #keyword 개수
    keyword_cnt=0
    for w in token_list:
        if w in keywords:
            keyword_cnt +=1
    return keyword_cnt * keyword_cnt / window_size

In [8]:
get_sentence_weight(['바나나','사과','바나나','바나나','포도'],{'사과', '포도'})

1.0

In [9]:
def summarize(context, no_sentences=5):
    word_list = get_words(context,isNoun=True) #기사에서 단어 가져오기
    keywords = get_keywords(word_list) #단어 목록에서 중요 키워드 뽑기
    
    sentence_list = get_sentences(context) #문장 토큰화해서 리스트로 가져오기
    
    sentence_weight =[]
    for sentence in sentence_list:
        token_list = get_words(sentence, isNoun = False) #안 써도 됨
        sentence_weight.append((get_sentence_weight(token_list,keywords), sentence)) #어떤 문장인지 보기위해 tuple로 넘겨줌
    
    sentence_weight.sort(reverse=True)
    
    return sentence_weight[:no_sentences]

In [10]:
sum_sents = summarize(news_article,3)
for s in sum_sents:
    print(s)

(26.07758620689655, '대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다.')
(21.333333333333332, '과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다.')
(18.753246753246753, '유영민 과기정통부 장관은 “오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리”라며 “세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만, 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다.')


textrank

In [11]:
text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나"

In [12]:
#유사도 함수_자카드 유사도=교집합/합집합
def sentence_similarity(sent1,sent2):
    
    sent1_list = [token[0] for token in mecab.pos(sent1) if token[1][0] in ['N','V']]
    sent2_list = [token[0] for token in mecab.pos(sent2) if token[1][0] in ['N','V']]
    
    union = set(sent1_list).union(set(sent2_list))
    intersection = set(sent1_list).intersection(set(sent2_list))
    
    return len(intersection)/len(union)

sentence_similarity("나는 치킨을 좋아해","나는 치킨을 싫어해")
#자카드유사도=2/(3+3-2)

0.5

In [21]:
import numpy as np

def buildMatrix(sentences):
    sentences_len = len(sentences)
    score = np.ones(sentences_len, dtype=np.float32)
    
    weighted_edge = np.zeros((sentences_len,sentences_len),dtype=np.float32)
    
    for i in range(sentences_len):
        for j in range(sentences_len):
            if i == j: #나와 나 사이의 유사도는 0으로 만들어줌
                continue
            
            weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])
    
    for i in range(sentences_len):
        score[i] = weighted_edge[i].sum() #해당 문장에 링크 걸려 있는 것들
        weighted_edge[i] /= score[i] #각각의 element를 sum한 걸로 나눠주는 것
        #0.5 / (0.5+0.167+0.8) ~~이런 식으로 엣지 가중치 업데이트
    
    return weighted_edge

buildMatrix(sent_tokenize(text))

array([[0.        , 0.3409091 , 0.11363637, 0.54545456],
       [0.45454544, 0.        , 0.        , 0.54545456],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142854, 0.4285714 , 0.        , 0.        ]], dtype=float32)

In [23]:
#score와 엣지 가중치 열끼리 곱한다음 더해서 d 적용해서 score 업데이트
#전에는 scoring할 때: summation 활용한 두가지 방법 // dot연산(행렬 곱)
def scoring(weighted_edge, score, eps=0.0001, d=0.85, max_iter=50): #eps=trheshold
    for it in range(max_iter):
        new_score = (1-d) + d * weighted_edge.T.dot(score) #score가 new score가 됨
        
        for diff in abs(new_score-score):
            if diff <= eps: #diff는 score 수만큼, eps=threshold이므로 최소 이 이상 돌리겠다
                return new_score
            
        score = new_score
    return score

In [70]:
#문장 요약 절차
def summize(text, no_sentences):
    sentence_list = sent_tokenize(text) #문장 토큰화
    weighted_edge = buildMatrix(sentence_list) #엣지 가중치 만들기
    score_init = np.ones(weighted_edge.shape[0], dtype=np.float32) #점수 초기값 #정방행렬이므로
    score = scoring(weighted_edge, score_init) #점수 업데이트
    sorted_score = sorted(enumerate(score), key=lambda x:x[1], reverse=True)[:no_sentences] #그냥 sorting하면 idx로 하게됨,so점수로 sorting하기 위해 x[1]
    return [(s[1],sentence_list[s[0]]) for s in sorted_score]

#my_점수와 문장 변환 코드
#     for i in range(len(sorted_score)):
#         print("점수: {}, 문장: {}".format(sorted_score[i][1],sentence_list[sorted_score[i][0]]))

In [71]:
summize(text,3) #상위3개 각 문장 점수와 문장

[(1.4098104, '딸기 바나나 사과 파인애플 수박.'),
 (1.2793452, '파인애플 사과 딸기 바나나'),
 (1.0245311, '바나나 사과 딸기 포도.')]

In [72]:
#extractive summarization 추출적 요약; 기존 문장에서 뽑는 것 ~~ 통계기반
#<=> abstractive summarization 추상적 요약; generate생성모델 ~~ 딥러닝
summize(news_article,3) #기사 3문장 요약

[(1.3107076,
  '과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다.'),
 (1.2891543,
  '유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다.'),
 (1.2184815,
  '10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다.')]